In [21]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
#import org.openqa.selenium.Keys
from random import randint
from time import sleep
import tldextract
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.proxy import Proxy, ProxyType

import pandas as pd
import logging
from chromedriver_py import binary_path

In [22]:
prox = Proxy()
prox.proxy_type = ProxyType.MANUAL
prox.http_proxy = "ip_addr:port"
prox.socks_proxy = "ip_addr:port"
prox.ssl_proxy = "ip_addr:port"

capabilities = webdriver.DesiredCapabilities.CHROME
prox.add_to_capabilities(capabilities)

drive_path = r"C:\Users\shankar\Desktop\projects\chromedriver.exe"
# keep clicking untill no click buttion
URL = 'https://www.tickertape.in/'
chrome_options = webdriver.ChromeOptions();
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--window-size=1280x1696')
chrome_options.add_argument('--user-data-dir=/tmp/user-data')
chrome_options.add_argument('--hide-scrollbars')
chrome_options.add_argument('--enable-logging')
chrome_options.add_argument('--log-level=0')
chrome_options.add_argument('--v=99')
chrome_options.add_argument('--single-process')
chrome_options.add_argument('--data-path=/tmp/data-path')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--homedir=/tmp')
chrome_options.add_argument('--disk-cache-dir=/tmp/cache-dir')
chrome_options.add_argument("user-agent= Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36")
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation']);

In [3]:
def news_tracker(url,drive_path,stock):
    # get https://www.tickertape.in/ web page
    def basic_url(url):
        chrome_options = webdriver.ChromeOptions();
        chrome_options.add_experimental_option("excludeSwitches", ['enable-automation']);
        driver= webdriver.Chrome(executable_path=binary_path,options=chrome_options)
        driver.get(url)
        driver.implicitly_wait(randint(1,5))
        return driver
    base_driver = basic_url(URL) 
    
    ########################
    # serch for compeny stockes
    def search_key(word,driver):
        search= driver.find_element_by_id("search-stock-input").send_keys(word) 
        driver.implicitly_wait(randint(1,4))
        act = ActionChains(driver)
        act.send_keys(Keys.ENTER).perform()
        driver.implicitly_wait(randint(1,4))
        driver.find_element_by_link_text('News').click()
        driver.implicitly_wait(randint(1,5))
        driver.find_element_by_class_name('suggestion-wrapper').click()
        driver.find_element_by_xpath('//*[@id="menu-items-list"]/ul/div[2]').click()
        return driver
    
    search_drive = search_key(stock,base_driver)
    
    #news_urls = search_drive.find_elements_by_xpath("//div[@data-section-tag='latestNews'or @data-section-tag='olderNews']/div/a")
    #for url in news_urls:
        #sours= url.find_element_by_xpath(".//div[2]/p/span[3]").text 
    return search_drive

In [4]:
search_drive = news_tracker(URL,drive_path,'Reliance Industries')
driver.implicitly_wait(randint(1,3))
while True:
    driver.implicitly_wait(randint(1,3))
    try:
        load_more = search_drive.find_element_by_xpath('//*[@id="load-more"]/button').click()
    except NoSuchElementException:
        break
news_urls = search_drive.find_elements_by_xpath("//div[@data-section-tag='latestNews'or @data-section-tag='olderNews']/div/a")

In [5]:
news_link =[]
for i in news_urls:
    link= i.get_property('href')
    ele = i
    ele_details = {'links':link,'ele':ele}
    news_link.append(ele_details)
news_urls_df = pd.DataFrame(news_link)

In [6]:
news_urls_df.to_csv(r'C:\Users\shankar\Desktop\projects\V2\news_urls_df.csv',index=False)

In [7]:
news_urls_df.shape

(3368, 2)

In [8]:
news_urls_df.ele[0].find_element_by_xpath(".//div[2]/h5/p").text

'Mukesh Ambani to appeal against SEBI penalty'

In [9]:
%cd C:\Users\shankar\Desktop\projects\V2

C:\Users\shankar\Desktop\projects\V2


In [10]:
def Economic_Times(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    dri.delete_all_cookies() 
    dri.refresh()
    driver.implicitly_wait(randint(2,5))
    try:
        news = dri.find_element_by_xpath('//*[@class="pageContent flt"]/article/div[2]').text
        news_date = dri.find_element_by_xpath('//*[@class="dt contentwrapper"]/div/time').text
    except:
        try:
            news =  dri.find_element_by_xpath('//*[@class="pageContent flt"]/article/div[3]').text 
            news_date = dri.find_element_by_xpath('//*[@class="dt contentwrapper"]/div/time').text
        except:
            news_date = ' '
            news = ' '
    #dri.close()
    return news_date,news 

In [11]:
def Business_Standard(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,5)) 
    try:
        news_date = dri.find_element_by_xpath("//*[@class= 'last-update mT10']/p/span[2]").text
        news = dri.find_element_by_xpath('//*[@id="hpcontentbox"]/div[5]/div[1]/div/div[3]/span').text
    except:
        try:
            news_date = dri.find_element_by_xpath("//*[@class= 'last-update mT10']/p/span[2]").text
            news = dri.find_element_by_xpath('//*[@id="hpcontentbox"]/div[5]/div[1]/div/div[5]/span').text
        except:
            try:
                news_date = dri.find_element_by_xpath("//*[@class= 'last-update mT10']/p/span[2]").text
                news = dri.find_element_by_xpath("//*[@class= 'p-content']").text
            except:
                news_date = ' '
                news = ' '            
    #dri.close()
    #data ={'date':date,'news_story':news}
    return news_date,news

In [12]:
def Hindu_Businessline(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(1,3)) 
    #head = dri.find_element_by_class_name('title-bor-bottom').text
    news_date = dri.find_element_by_xpath("//*[@class ='update-date']/none").text
    news = dri.find_element_by_xpath('//*[@id="ControlPara"]/div[1]').text
    #dri.close()
    return news_date,news

In [13]:
def Business_Today(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,4)) 
    #head = dri.find_element_by_class_name('headline').text
    news_day = dri.find_element_by_xpath('//*[@class="story-details"]').text
    news =     dri.find_element_by_xpath('//*[@class="story-right relatedstory paywall"]').text
    #dri.close()
    return news_day,news

In [14]:
def Moneycontrol(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,5)) 
    try:
        news_date = dri.find_element_by_xpath("//*[contains(@id,'article')]/div[5]/div[2]").text
        news = dri.find_element_by_xpath("//*[contains(@id,'article')]/div[3]").text
    except:
        news_date = dri.find_element_by_xpath("//*[contains(@id,'article')]/div[1]/div[2]").text
        news = dri.find_element_by_xpath("//*[contains(@id,'article')]/div[3]").text
    #dri.close()
    return news_date,news 

In [15]:
def Free_Press_Journal(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    dri.delete_all_cookies()
    dri.refresh()
    driver.implicitly_wait(randint(2,5)) 
    date = dri.find_element_by_xpath("//*[@id='container']/div/div[1]/div/div[1]/div[1]/div[1]/div[1]/time").text
    news = dri.find_element_by_xpath('//*[@id="container"]/div/div[1]/div/div[1]/div[1]/div[5]/div[2]/div[2]/div/div[1]/div[1]/div[1]/div/div').text
    #dri.close()
    #data ={'date':date,'news_story':news}
    return date,news

In [16]:
def Livemint(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    dri.delete_all_cookies() 
    dri.refresh()
    driver.implicitly_wait(randint(2,5)) 
    news_day = dri.find_element_by_xpath("//*[contains(@id,'tBox')]").text
    news_1 = dri.find_element_by_xpath("//*[contains(@id,'summary')]").text
    news_2 = dri.find_element_by_xpath("//*[contains(@id,'mainArea')]/div[1]").text
    news_3 = dri.find_element_by_xpath("//*[contains(@id,'mainArea')]/div[2]").text
    news = " ".join([news_1, news_2,news_3]) 
    #dri.close()
    return news_day,news

In [17]:
def Financial_Express(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,5))
    news_day = dri.find_element_by_xpath('//*[@class="storybox" or @class="story_mid"]/div/div/div/span').text 
    try:
        news = dri.find_element_by_xpath('//*[@id="main-story-1"]/div/div[1]/div[2]/div/div[2]').text
    except:
        news = dri.find_element_by_xpath("//*[@class='storybox']/div[2]/div").text
    #dri.close()
    return news_day,news

In [18]:
def India_Infoline(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,3)) 
    #head = dri.find_element_by_xpath('//*[@class="card primary_contnent_bg story_card_temp"]/section/header/h2').text
    news_day = dri.find_element_by_xpath('//*[@class="card primary_contnent_bg story_card_temp"]/section/header/div/div/p/span').text
    news = dri.find_element_by_xpath('//*[@id="story_card"]/div[2]').text
    #dri.close()
    return news_day,news

In [19]:
def Firstpost(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,4))
    #head = dri.find_element_by_xpath('//*[@id="headlineitem"]').text
    news_day= dri.find_element_by_xpath('//*[@class="author-info"]/span').text
    news = dri.find_element_by_xpath('//*[@class="inner-copy article-full-content"]').text
    #dri.close()
    return news_day,news

In [20]:
def News18(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,5)) 
    #head = dri.find_element_by_xpath('//*[@id="left"]/div[3]/h1').text
    news_day = dri.find_element_by_xpath('//*[@id="left"]/div[position()= 2 or position()= 3]/div[1]/div[2]/ul/li[2]').text
    news = dri.find_element_by_xpath('//*[@id="left"]/div[position()= 2 or position()= 3]/article').text
    #dri.close()
    return news_day,news

In [21]:
def Bloomberg_Quint(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,5))
    try:
        #head = dri.find_element_by_xpath("//*[contains(@class,'container story-base-template')]/section/div[1]/h1").text
        news_day = dri.find_element_by_xpath("//*[contains(@class,'container story-base-template')]/section/div[1]/div[2]/div/time").text
        news = dri.find_element_by_xpath("//*[contains(@id,'piano-content-controller')]").text
    except:
        news_date = ' '
        news = ' '
    #dri.close()
    return news_day,news

In [22]:
def The_Hindu(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,4)) 
    #head = dri.find_element_by_xpath("//*[@class='article']/div/h1").text
    news_day= dri.find_element_by_xpath("//*[@class='article']/div/div[2]/div/div/span").text
    news = dri.find_element_by_xpath("//*[contains(@id,'content-body')]").text
    #dri.close()
    return news_day,news

In [23]:
  def Investing_com(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,4)) 
    try:
        head = dri.find_element_by_tag_name("article")
        #headline= head.find_element_by_class_name('article-item-title').text
        #headline = head.find_element_by_xpath("//header/h1").text
        news_date=    head.find_element_by_xpath("//header/footer").text
        news =        head.find_element_by_class_name('article-item-content-wrapper').text
    except:
        news_date = ' '
        news = ' '
    #dri.close()
    return news_date,news

In [24]:
def Reuters(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(1,3)) 
    #head = dri.find_element_by_xpath('//*[@id="__next"]/div/div[3]/div[2]/div/div/h1').text
    raw_data = dri.find_element_by_class_name('ArticleHeader-date-line-3oc3Y')
    day =  raw_data.find_element_by_xpath('//time[1]').text
    time =  raw_data.find_element_by_xpath('//time[2]').text
    data = " ".join([day,time])     
    news = dri.find_element_by_class_name('ArticleBodyWrapper').text 
    #dri.close()
    return data,news

In [25]:
def Hindustan_Times(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(2,5)) 
    #head = dri.find_element_by_xpath("//*[@id='dataHolder']/div[1]/div[1]/h1").text
    date = dri.find_element_by_xpath("//*[@id='dataHolder']/div[1]/div[1]/div[4]/div[2]").text
    news = dri.find_element_by_xpath("//*[@class='detail']").text
    #dri.close()
    return date,news

In [26]:
def zeenews(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    #dri.delete_all_cookies() 
    driver.implicitly_wait(randint(1,4)) 
    dri.find_element_by_xpath('//*[@class="tbl-read-more-box"]/a').click()
    try:
        news_date = dri.find_element_by_xpath("//*[@class = 'article-share-block margin-bt30px inner static']/div[6]").text
    except:
        news_date = dri.find_element_by_xpath("//*[@class = 'article-share-block margin-bt30px inner static']/div[5]").text
    news = dri.find_elements_by_xpath('//*[@class="field-items"]/div')[-1].text
    return news_date,news

In [27]:
def Indian_Express(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    #dri.delete_all_cookies() 
    driver.implicitly_wait(randint(1,4)) 
    try:
        news_date = dri.find_element_by_xpath('//*[@id="storycenterbyline"]/span').text
        news_div = dri.find_elements_by_xpath("//*[@id='pcl-full-content']/p")
        news = []
        x = [news.append(i.text) for i in news_div]
        news = ' '.join(news)
    except:
        news_date = dri.find_element_by_xpath('//*[@class="ie-authorbox"]/div/span').text
        news_div = dri.find_elements_by_xpath('//*[@class="articles"]/div/p')
        news = []
        x = [news.append(i.text) for i in news_div]
        news = ' '.join(news)
    return news_date,news

In [19]:
def aninews(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    dri.delete_all_cookies()
    dri.refresh()
    driver.implicitly_wait(randint(1,3))
    #head = dri.find_element_by_xpath("//*[@id='dataHolder']/div[1]/div[1]/h1").text
    date = dri.find_element_by_xpath("//*[@id='news-detail-block']/div[1]/article/div/p/span[2]").text
    news = dri.find_element_by_xpath("//*[@id='news-detail-block']/div[1]/article/div[2]").text
    #dri.close()
    return date,news

In [ ]:
def outlookindia(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    driver.implicitly_wait(randint(1,4)) 
    #head = dri.find_element_by_xpath("//*[@id='dataHolder']/div[1]/div[1]/h1").text
    date = dri.find_element_by_xpath("//*[@class ='author_name_date']").text
    news_div = dri.find_elements_by_xpath("//*[@id='articleBody']/p")
    news = []
    x = [news.append(i.text) for i in news_div]
    news = ' '.join(news)
    #dri.close()
    return date,news

In [29]:
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

formatter = logging.Formatter('%(levelname)s:%(name)s:%(message)s')

file_handler = logging.FileHandler('url_data.log')
file_handler.setLevel(logging.ERROR)
file_handler.setFormatter(formatter)

stream_handler = logging.StreamHandler()
stream_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(stream_handler)


dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)

Error_url =[]
news_data = []
for url in news_urls_df.ele:
    headline = url.find_element_by_xpath(".//div[2]/h5/p").text
    publisher= url.find_element_by_xpath(".//div[2]/p/span[3]").text
    link = url.get_property('href')
    sours = tldextract.extract(link).domain
    try:
        if sours == "indiatimes":
            news_day,news = Economic_Times(link)
        elif sours == "business-standard":
            news_day,news= Business_Standard(link)
        elif sours == "thehindubusinessline":
            news_day,news= Hindu_Businessline(link)
        elif sours == "businesstoday":
            news_day,news= Business_Today(link)
        elif sours == "moneycontrol":
            news_day,news= Moneycontrol(link)
        elif sours == "freepressjournal":
            news_day,news= Free_Press_Journal(link) 
        elif sours == "livemint":
            news_day,news= Livemint(link)
        elif sours == "financialexpress":
            news_day,news= Financial_Express(link)
        elif sours == "indiainfoline":
            news_day,news= India_Infoline(link)
        elif sours == "firstpost":
            news_day,news= Firstpost(link)
        elif sours == "news18":
            news_day,news= News18(link)
        elif sours == "bloombergquint":
            news_day,news= Bloomberg_Quint(link)
        elif sours == "thehindu":
            news_day,news= The_Hindu(link)
        elif sours == "investing":
            news_day,news= Investing_com(link)
        elif sours == "reuters":
            news_day,news= Reuters(link)
        elif sours == "hindustantimes":
            news_day,news=Hindustan_Times(link)
        elif sours == "india":
            news_day,news=zeenews(link)
        elif sours == "indianexpress":
        elif sours == "aninews":
            news_day,news=aninews(url)
        else:
            news_day = 'NO date'
            news =    'NO News'
    except:
        error_details = {'headlines':headline,'sources':sours,'links':link}
        Error_url.append(error_details)
        logger.exception('no such element Unable to locate element')
        continue
    else:
        details = {'headlines':headline,'sources':sours,'links':link,'date':news_day,'news_story':news }
        news_data.append(details) 
dri.close()

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-14-0456c693b30c>", line 6, in Moneycontrol
    news_date = dri.find_element_by_xpath("//*[contains(@id,'article')]/div[5]/div[2]").text
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 976, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Mess

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-29-63d9fe2ec3cd>", line 50, in <module>
    news_day,news= Bloomberg_Quint(link)
  File "<ipython-input-21-4eab8d445c13>", line 13, in Bloomberg_Quint
    return news_day,news
UnboundLocalError: local variable 'news_day' referenced before assignment
ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-29-63d9fe2ec3cd>", line 62, in <module>
    news_day,news=Indian_Express(url)
  File "<ipython-input-27-5b284378b2e6>", line 3, in Indian_Express
    dri.get(url)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\shankar\anaconda3\lib\si

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-14-0456c693b30c>", line 6, in Moneycontrol
    news_date = dri.find_element_by_xpath("//*[contains(@id,'article')]/div[5]/div[2]").text
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 976, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.NoSuchElementException: Mess

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-29-63d9fe2ec3cd>", line 60, in <module>
    news_day,news=zeenews(link)
  File "<ipython-input-26-cd448b881be6>", line 6, in zeenews
    dri.find_element_by_xpath('//*[@class="tbl-read-more-box"]/a').click()
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 976, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, scr

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-29-63d9fe2ec3cd>", line 60, in <module>
    news_day,news=zeenews(link)
  File "<ipython-input-26-cd448b881be6>", line 6, in zeenews
    dri.find_element_by_xpath('//*[@class="tbl-read-more-box"]/a').click()
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 394, in find_element_by_xpath
    return self.find_element(by=By.XPATH, value=xpath)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 976, in find_element
    return self.execute(Command.FIND_ELEMENT, {
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, scr

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-29-63d9fe2ec3cd>", line 64, in <module>
    news_day,news=aninews(url)
  File "<ipython-input-28-937383dc5db0>", line 3, in aninews
    dri.get(url)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.InvalidArgumentException: Message: invalid argument: 'url' must be a string
  (Session info: chrome=89.0.4389.114)

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  Fil

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  File "<ipython-input-29-63d9fe2ec3cd>", line 64, in <module>
    news_day,news=aninews(url)
  File "<ipython-input-28-937383dc5db0>", line 3, in aninews
    dri.get(url)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 333, in get
    self.execute(Command.GET, {'url': url})
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py", line 321, in execute
    self.error_handler.check_response(response)
  File "C:\Users\shankar\anaconda3\lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 242, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.InvalidArgumentException: Message: invalid argument: 'url' must be a string
  (Session info: chrome=89.0.4389.114)

ERROR:__main__:no such element Unable to locate element
Traceback (most recent call last):
  Fil

In [43]:
news_df= pd.DataFrame(news_data)

In [45]:
news_df.to_csv(r'C:\Users\shankar\Desktop\projects\reliance.CSV',index=False)

In [46]:
error_df= pd.DataFrame(Error_url)

In [48]:
error_df.to_csv(r'C:\Users\shankar\Desktop\projects\reliance_df_error.CSV',index=False)
error_df.shape

(51, 3)

In [49]:
news_df.head()

,headlines,sources,links,date,news_story
0,Mukesh Ambani to appeal against SEBI penalty,freepressjournal,https://www.freepressjournal.in/business/mukes...,": Thursday, April 8, 2021, 11:19 PM IST",Reliance Industries Chairman Mukesh Ambani wil...
1,RIL promoters to appeal against SEBI penalty,thehindubusinessline,https://www.thehindubusinessline.com/markets/r...,"April 08, 2021",Promoters of Reliance Industries Ltd will file...
2,Mukesh Ambani to appeal against Rs 25-crore pe...,business-standard,https://www.business-standard.com/article/comp...,"Last Updated at April 8, 2021 23:23 IST",Reliance Industries Chairman Mukesh Ambani wil...
3,Reliance Industries shares shrug off Sebi fine...,livemint,https://www.livemint.com/market/mark-to-market...,"Updated: 08 Apr 2021, 11:19 AM IST",Consumer-oriented businesses such as telecom a...
4,Sebi slaps Rs 25 crore fine on Ambani brothers...,freepressjournal,https://www.freepressjournal.in/business/sebi-...,": Thursday, April 8, 2021, 8:05 AM IST",Sebi on Wednesday imposed a total penalty of R...


In [51]:
news_df[news_df['date']==' '].shape

(68, 5)

In [52]:
news_df[news_df['date']=='NO date']

,headlines,sources,links,date,news_story
109,Sensex Tanks 397 Pts; Nifty Slips Below 15K,outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
121,Sensex Collapses 487 Points; Nifty Tanks Below...,outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
384,"Sensex Sets New Records, Hits 50,000-Mark Ahea...",outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
395,Sensex Rallies Nearly 500 Points In Early Trad...,outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
461,"Seven Of Top-10 Firms Add Rs 75,845 Crore Mark...",outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
464,Reliance Petroleum: SEBI Imposes A Fine Of Rs ...,outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
498,"6 Firms Added Rs 60,198 Cr In Market Capital T...",outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
600,Five Of Top 10 Most Valued Firms Lose Together...,outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
984,"Eyeing Upcoming Festive Sales, Amazon Infuses ...",outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News
1164,"After Jio Investment, Silver Lake Picks 1.75% ...",outlookindia,https://www.outlookindia.com/website/story/bus...,NO date,NO News


In [55]:
news_df[news_df['date']=='NO date'].links.iloc[3]

'https://www.outlookindia.com/website/story/business-news-sensex-rallies-nearly-500-points-in-early-trade-nifty-tops-14400/371067'

In [66]:
def outlookindia(url):
    #dri= webdriver.Chrome(executable_path=drive_path,options=chrome_options)
    dri.get(url)
    dri.delete_all_cookies()
    dri.refresh()
    sleep(randint(1,3))
    #head = dri.find_element_by_xpath("//*[@id='dataHolder']/div[1]/div[1]/h1").text
    date = dri.find_element_by_xpath("//*[@class ='author_name_date']").text
    news_div = dri.find_elements_by_xpath("//*[@id='articleBody']/p")
    news = []
    x = [news.append(i.text) for i in news_div]
    news = ' '.join(news)
    #dri.close()
    return date,news

In [23]:
dri= webdriver.Chrome(executable_path=drive_path,desired_capabilities=capabilities,options=chrome_options)

InvalidArgumentException: Message: invalid argument: cannot parse capability: proxy
from invalid argument: Specifying 'socksProxy' requires an integer for 'socksVersion'


In [18]:
outlookindia('https://www.outlookindia.com/website/story/business-news-sensex-rallies-nearly-500-points-in-early-trade-nifty-tops-14400/371067')

('PTI\n19 January 2021',
 'Equity benchmark Sensex rallied nearly 500 points in early trade on Tuesday tracking strong gains in index majors Reliance Industries, HDFC and Infosys amid a positive trend in global markets. The 30-share BSE index was trading 485.97 points or 1 per cent higher at 49,050.24. Similarly, the broader NSE Nifty surged 134.05 points or 0.94 per cent to 14,415.35. Bajaj Finance was the top gainer in the Sensex pack, rising around 2 per cent, followed by SBI, ONGC, IndusInd Bank, Reliance Industries, HDFC and Axis Bank. On the other hand, ITC and HDFC Bank were the laggards. In the previous session, Sensex ended 470.40 points or 0.96 per cent lower at 48,564.27, while Nifty tumbled 152.40 points or 1.06 per cent to 14,281.30. Foreign portfolio investors (FPIs) were net buyers in the capital market as they purchased shares worth Rs 650.60 crore on Monday, as per exchange data. According to Binod Modi Head-Strategy at Reliance Securities, domestic equities look to be

In [20]:
aninews('https://www.aninews.in/news/world/others/morocco-receives-covid-19-vaccine-through-covax20210409074013/')

('Apr 09, 2021 07:40',
 "Rabat [Morocco], April 9 (ANI/Xinhua): Morocco on Thursday received the first batch of COVID-19 vaccines through the global vaccine-sharing facility COVAX, the Ministry of Health said.\nAdditional doses will be allocated to the country in the coming weeks, said the ministry in a statement, without giving further details about the vaccine Morocco received under this program.\nThe country launched a nationwide vaccination campaign at the end of January, after the arrival of the first shipment of China's Sinopharm vaccines.\nSPONSORED CONTENT\nA Girl From Bengaluru Became A Billionaire Using This Method\nOlymp Trade\nEarn Money Simply! Trading Strategy For Housewives In India\nOlymp Trade\nBengaluru Woman Woke Up Rich After An Unexpected Mistake\nEuropa Casino\nStruggling Families Lift Themselves Out Of Poverty In Bengaluru\nOlymp Trade\nAs of Thursday, 4,433,939 people in the country have received the first shot, and 4,075,290 have received the second dose.\nCOVI